`Biblioteki`

In [1]:
import math
import os
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np

from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from hurry.filesize import size
import time

`Okienko`

In [3]:
import tkinter as tk
from tkinter import messagebox
from PIL import ImageTk, Image
from utils_new import *
tekst = """Proszę uzupełnić poniższe pola. Jeśli chcesz wpisać więcej wartości to zrób to używając przecinków. Rok i nazwa pliku
są obligatoryjne. Docelowo są to 2 pliki csv, ale zamiast nich możesz też wygenerować jeden plik excel zaznaczając odpowiedni 
checkbox. Plik pojawi się po dłuższej chwili w ścieżce (...). Gdy plik będzie gotowy pojawi się stosowne okienko. By utworzyć 
plik naciśnij przycisk 'Wykonaj'."""
def execute_function():
    # Pobierz wartości z pól
    rok = entry_rok.get().replace(" ", "").split(',')
    if rok == ['']:
        messagebox.showerror("Błąd", "Pole 'Rok' musi zostać wypełnione")
        return
    
    miesiac = entry_miesiac.get().replace(" ", "").split(',')
    if len(miesiac) == 0:
        miesiac = [i for i in range(1,13)]
    else:
        pass
    
    kolumny = entry_kolumny.get().replace(" ", "").split(',')
    if kolumny == ['']:
        kolumny=None
    else:
        pass
    
    kontrahenci = entry_numery_kontrahentow.get().replace(" ", "").split(',')
    if len(kontrahenci) == ['']:
        kontrahenci = None
    else:
        pass
    
    nazwa_pliku = entry_nazwa_pliku.get()
    if not nazwa_pliku:
        messagebox.showerror("Błąd", "Musisz wprowadzić nazwę pliku")
        return
    
    # Warunki równościowe na wartości rekordów
    s_wieksza_niz = entry_s_w_r.get()
    if len(s_wieksza_niz) == 0:
        s_wieksza_niz = float('-inf')
    else:
        s_wieksza_niz = float(s_wieksza_niz)
        
    s_mniejsza_niz = entry_s_m_r.get()
    if len(s_mniejsza_niz) == 0:
        s_mniejsza_niz = float('inf')
    else:
        s_mniejsza_niz = float(s_mniejsza_niz)
        
    z_wieksza_niz = entry_z_w_r.get()
    if len(z_wieksza_niz) == 0:
        z_wieksza_niz = float('-inf')
    else:
        z_wieksza_niz = float(z_wieksza_niz)
        
    z_mniejsza_niz = entry_z_m_r.get()
    if len(z_mniejsza_niz) == 0:
        z_mniejsza_niz = float('inf')
    else:
        z_mniejsza_niz = float(z_mniejsza_niz)
    
    # Wywołaj funkcję wygeneruj_plik z przekazanymi wartościami
    try:
        ts,tz = funkcja_generująca_plik(rok=rok, miesiac=miesiac, kolumny=kolumny, kontrahenci=kontrahenci, nazwa_pliku=nazwa_pliku, czarna_lista = czarna_lista_var.get(),
                                        zagranica = zagraniczne_podmioty_var.get(), excel = excel_var.get(), s_wieksza_niz = s_wieksza_niz, s_mniejsza_niz = s_mniejsza_niz, 
                                        z_wieksza_niz = z_wieksza_niz, z_mniejsza_niz = z_mniejsza_niz)
    except Exception as e:
        messagebox.showerror("Błąd", "Wystąpił błąd: " + str(e) + "\nSkontaktuj się z twórcą programu (spójrz na prawy dolny róg)")
    # Wyświetl komunikat
    if (ts != 0) & (tz != 0):
        messagebox.showinfo("Komunikat", "Pliki sprzedażowe i zakupowe gotowe!")
    elif (ts == 0) & (tz != 0):
        messagebox.showinfo("Komunikat", "Został wygenerowany plik zakupowy!")
    elif (ts != 0) & (tz == 0):
        messagebox.showinfo("Komunikat", "Został wygenerowany plik sprzedażowy!")
    else:
        messagebox.showinfo("Komunikat", "Żaden plik nie został wygenerowany. Brak rekordów!")
    
def justify_text(text, width):
    words = text.split() # podział tekstu na słowa
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + word) + 1 <= width: # sprawdzenie czy słowo zmieści się w bieżącej linii
            if current_line:
                current_line += " "
            current_line += word
        else:
            lines.append(current_line)
            current_line = word
    lines.append(current_line)
    justified_text = "\n".join(lines)
    return justified_text

# Tworzenie okna
root = tk.Tk()
root.title("JPK Column Search")
root.iconbitmap('start.ico')
root.geometry("650x320")

# Ramka na górze
top_frame = tk.Frame(root)
top_frame.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Etykieta na górze
label_top = tk.Label(top_frame, text=justify_text(tekst, 120), font=("Helvetica", 10), justify="left")
label_top.grid(row=0, column=0, columnspan = 3, padx=5, pady=5)

# Zdjęcie na lewej stronie
image = Image.open("start2.jpg") #start2
resize_image = image.resize((375, 225))#250,150
my_img = ImageTk.PhotoImage(resize_image)
my_label = tk.Label(image=my_img)
my_label.grid(row=1, column=0, padx=15)

# Ramka na prawej stronie
right_frame = tk.Frame(root)
right_frame.grid(row=1, column=1, padx=10, pady=10)

# Etykiety i pola na prawej stronie
label_1 = tk.Label(right_frame, text="Pola, które koniecznie należy uzupełnić", font=("Arial", 9, "bold"))
label_1.grid(row=0, column=0, columnspan=2, padx=20, sticky = tk.E)

label_rok = tk.Label(right_frame, text="Rok:", font = ('TkDefaultFont', 9, 'underline'))
label_rok.grid(row=1, column=0, sticky=tk.E)
entry_rok = tk.Entry(right_frame)
entry_rok.grid(row=1, column=1)

label_miesiac = tk.Label(right_frame, text="Miesiąc:")
label_miesiac.grid(row=2, column=0, sticky=tk.E)
entry_miesiac = tk.Entry(right_frame)
entry_miesiac.grid(row=2, column=1)

label_kolumny = tk.Label(right_frame, text="Kolumny:")
label_kolumny.grid(row=3, column=0, sticky=tk.E)
entry_kolumny = tk.Entry(right_frame)
entry_kolumny.grid(row=3, column=1)

label_numery_kontrahentow = tk.Label(right_frame, text="Numery_kontrahentów (NIP):")
label_numery_kontrahentow.grid(row=4, column=0, sticky=tk.E)
entry_numery_kontrahentow = tk.Entry(right_frame)
entry_numery_kontrahentow.grid(row=4, column=1)

label_nazwa_pliku = tk.Label(right_frame, text="Nazwa pliku:", font = ('TkDefaultFont', 9, 'underline'))
label_nazwa_pliku.grid(row=5, column=0, sticky=tk.E)
entry_nazwa_pliku = tk.Entry(right_frame)
entry_nazwa_pliku.grid(row=5, column=1)

label_2 = tk.Label(right_frame, text="Warunki wartościowe na rekordy dotyczące sprzedaży", font=("Arial", 9, "bold"))
label_2.grid(row=6, column=0, columnspan=2, padx=10, pady=10, sticky = tk.W)

label_s_w_r = tk.Label(right_frame, text="Większe niż lub równe :")
label_s_w_r.grid(row=7, column=0, sticky=tk.E)
entry_s_w_r = tk.Entry(right_frame)
entry_s_w_r.grid(row=7, column=1)

label_s_m_r = tk.Label(right_frame, text="Mniejsze niż lub równe :")
label_s_m_r.grid(row=8, column=0, sticky=tk.E)
entry_s_m_r = tk.Entry(right_frame)
entry_s_m_r.grid(row=8, column=1)

label_3 = tk.Label(right_frame, text="Warunki wartościowe na rekordy dotyczące zakupów", font=("Arial", 9, "bold"))
label_3.grid(row=9, column=0, columnspan=2, padx=10, pady=10, sticky = tk.W)

label_z_w_r = tk.Label(right_frame, text="Większe niż lub równe :")
label_z_w_r.grid(row=10, column=0, sticky=tk.E)
entry_z_w_r = tk.Entry(right_frame)
entry_z_w_r.grid(row=10, column=1)

label_z_m_r = tk.Label(right_frame, text="Mniejsze niż lub równe :")
label_z_m_r.grid(row=11, column=0, sticky=tk.E)
entry_z_m_r = tk.Entry(right_frame)
entry_z_m_r.grid(row=11, column=1)

# Checkboxy

# Zagraniczne podmioty
zagraniczne_podmioty_var = tk.BooleanVar()
zagraniczne_podmioty_var.set(False)

checkbutton_zagraniczne_podmioty = tk.Checkbutton(right_frame, text="Pokaż rekordy dla podmiotów z zagranicy", variable=zagraniczne_podmioty_var)
checkbutton_zagraniczne_podmioty.grid(row=12, column=0, columnspan=2, padx=2, pady=2, sticky = tk.W)

# Wybór zapisu excel vs csv
excel_var = tk.BooleanVar()
excel_var.set(False)

checkbutton_excel = tk.Checkbutton(right_frame, text="Chcę otrzymać wyniki w pliku excel*", variable=excel_var)
checkbutton_excel.grid(row=13, column=0, columnspan=2, padx=2, pady=2, sticky = tk.W)

label_4 = tk.Label(right_frame, text="*W przeciwnym razie zostaną wygenerowane 2 pliki w formacie .csv", font=("Arial", 8))
label_4.grid(row=14, column=0, columnspan=2, padx=10, pady=10, sticky = tk.W)

# Przycisk na prawej stronie
button_execute = tk.Button(right_frame, text="Wykonaj", command=execute_function, padx=20, pady=10, bg="green")
button_execute.grid(row=15, column=0, columnspan=2, padx=10, pady=10)


status_bar = tk.Label(root, text="© Copyright KF", bd=1, relief=tk.SUNKEN, anchor=tk.E)
status_bar.grid(row=16, column=0, columnspan=2, sticky=tk.W+tk.E)
root.minsize(width=795, height=620)
root.maxsize(width=795, height=620)
# Uruchomienie pętli głównej
root.mainloop()